<a href="https://colab.research.google.com/github/nklsrmy/CCADMACL_EXERCISES_COM222ML/blob/main/Revised%20Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 1

Use all feature selection methods to find the best features

## Dataset Information

## Features

Number of Instances: 20640

Number of Attributes: 8 numeric, predictive attributes and the target

Attribute Information:

MedInc - median income in block group

HouseAge - median house age in block group

AveRooms - average number of rooms per household

AveBedrms - average number of bedrooms per household

Population - block group population

AveOccup - average number of household members

Latitude - block group latitude

Longitude - block group longitude

## Target
The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

In [78]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

In [79]:
housing = fetch_california_housing()
df_data =  pd.DataFrame(housing.data, columns=housing.feature_names)
df_target = pd.DataFrame(housing.target, columns=["target"])

1. Use any filter method to select the best features

In [80]:
# put your answer here
from sklearn.feature_selection import mutual_info_regression

In [81]:
threshold = 4
high_score_features = []
df_housing_data = df_data.copy()
df_housing_target = df_target.copy()

feature_scores = mutual_info_regression(df_housing_data, df_housing_target)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [82]:
for score, r_name in sorted(zip(feature_scores, df_housing_data.columns), reverse=True)[:threshold]:
    high_score_features.append(r_name)

df_housing_fr = df_housing_data[high_score_features]
df_housing_fr.columns

Index(['Longitude', 'MedInc', 'Latitude', 'AveRooms'], dtype='object')

2. Use any wrapper method to select the best features

In [83]:
# put your answer here
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

rf_model = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
selector = RFE(rf_model, n_features_to_select=4, step=1)

selector = selector.fit(df_housing_data, df_housing_target.values.ravel())
selector_ind = selector.get_support()
df_housing_rfe = df_housing_data.iloc[:, selector_ind]
df_housing_rfe.columns

Index(['MedInc', 'AveRooms', 'AveOccup', 'Latitude'], dtype='object')

3. Use any embedded methood to select the best features

In [143]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_housing_data, df_housing_target, test_size=0.2, random_state=42)

In [148]:
# put your answer here
from sklearn.feature_selection import SelectFromModel

rf_model = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
rf_model = rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
sfm_rmse = mean_squared_error(y_test, sfm_preds, squared=False)

sel_sfm = SelectFromModel(rf_model, prefit=True)
sel_sfm_index = sel_sfm.get_support()
df_housing_sfm = df_housing_data.iloc[:, sel_sfm_index]
df_housing_sfm.columns

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Index(['MedInc', 'AveOccup'], dtype='object')

# Model Comparison

In [149]:
from sklearn.metrics import mean_squared_error, r2_score

In [150]:
model = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=3, n_estimators=500, random_state=0)

In [151]:
X_train_fr, X_test_fr, y_train_fr, y_test_fr = train_test_split(df_housing_fr, df_housing_target, test_size=0.2, random_state=42)

In [152]:
model_fr = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
model_fr.fit(X_train_fr, y_train_fr)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=3, n_estimators=500, random_state=0)

In [153]:
X_train_rfe, X_test_rfe, y_train_rfe, y_test_rfe = train_test_split(df_housing_rfe, df_housing_target, test_size=0.2, random_state=42)

In [154]:
model_rfe = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
model_rfe.fit(X_train_rfe, y_train_rfe)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=3, n_estimators=500, random_state=0)

In [157]:
default_preds = model.predict(X_test)
fr_preds = model_fr.predict(X_test_fr)
rfe_preds = model_rfe.predict(X_test_rfe)

In [158]:
default_rmse = mean_squared_error(y_test, default_preds, squared=False)
fr_rmse = mean_squared_error(y_test, fr_preds, squared=False)
rfe_rmse = mean_squared_error(y_test, rfe_preds, squared=False)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [159]:
print(f'Default RMSE: {default_rmse}')
print(f'Filter RMSE: {fr_rmse}')
print(f'Wrapper RMSE: {rfe_rmse}')
print(f'Embedded RMSE: {sfm_rmse}')

Default RMSE: 0.7750447841432082
Filter RMSE: 0.8166887384797389
Wrapper RMSE: 0.7753135462465947
Embedded RMSE: 0.7823934576498581
